In [ ]:
# default_exp parser

# Parser
> Este módulo processa o arquivo bin e extrai os metadados e dados do espectro dos blocos, além de criar estatísticas das medições.
  en: This module process the bin file extracting its metadata and spectrum levels besides extracting useful statistics.
  fr: Ce module traite le fichier bin et extrait les métadonnées et les données spectrales des blocs, en plus de créer des statistiques de mesure.

In [ ]:
#export
import sys, os
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

In [ ]:
%load_ext autoreload
%load_ext line_profiler
%load_ext cython
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [ ]:
#exporti
import os
import gc
from dataclasses import dataclass
from pathlib import Path
from typing import *
from collections import defaultdict
from dataclasses import asdict, make_dataclass
from fastcore.utils import parallel
from fastcore.foundation import L, GetAttr
from rfpye.constants import *
from rfpye.blocks import MAIN_BLOCKS, BaseBlock
from rfpye.utils import get_files, getattrs, bin2int, bin2str, cached
from rfpye.cyparser import cy_extract_compressed
from loguru import logger
import pandas as pd
import numpy as np
from rich import print

# For scripts
config = {
    "handlers": [
        {
            "sink": "parser.log",
            "serialize": True,
            "rotation": "1 month",
            "compression": "zip",
            "backtrace": True,
            "diagnose": True,
        },
    ],
}
logger.configure(**config)

[15]

In [ ]:
file = r"binfiles\v4\rfeye002159_SLMA_bimestral_PEAK_210313_120302.bin"

## Processamento do Arquivo `.bin` e criação dos diferentes tipos de blocos

In [ ]:
#export
def evaluate_checksum(file, next_block, data_size) -> int:
    """Receives a byte_block and verify if the calculated checksum is equal to the one registed in the specific byte"""
    start = file.tell()
    try:
        checksum = np.frombuffer(file.read(4), np.uint32).item()
    except ValueError:
        logger.error(f"Erro na leitura do checksum")
        file.seek(4, 1)
        return None
    block_size = file.tell() - next_block
    file.seek(-block_size, 1) # Go back to the beginning of the block
    calculated_checksum = (
            np.frombuffer(file.read(12+data_size), dtype=np.uint8)
            .sum()
            .astype(np.uint32)
            .item()
        )
    file.seek(4,1) # skip checksum
    if checksum != calculated_checksum:
        logger.error(f"Checksum diferente: {checksum} != {calculated_checksum}. Posicao: {file.tell()}")
        return None
    return checksum

In [ ]:
#export
def buffer2base_block(file, next_block: int) -> Union[BaseBlock, None]:
    """Receives an opened file buffer from the bin file and returns a dataclass with the attributes
    'thread_id', 'size', 'type', 'data', 'checksum' or None in case any error is identified.
    """
    thread_id = np.frombuffer(file.read(4), np.int32).item()
    block_size = np.frombuffer(file.read(4), np.int32).item()
    block_type = np.frombuffer(file.read(4), np.int32).item()
    data_block = file.read(block_size)
    if (checksum := evaluate_checksum(file, next_block, block_size)) is None:
        return None, None           
    return block_type, BaseBlock(thread_id, block_size, block_type, data_block, checksum)

A função a seguir recebe os bytes lidos do arquivo `.bin` e mapeia esses bytes em diferentes classes de acordo com o tipo de bloco

In [ ]:
#export
def create_block(file, next_block) -> Tuple:
    """Receives a byte_block, and converts it into one of the main classes
    Args: byte_block: A byte block directly returned from the file
    Returns: The Instance of the Block Type or None in case of error
    """
    block_type, base_block = buffer2base_block(file, next_block)
    if block_type is None:
        return None, None
    constructor = MAIN_BLOCKS.get(block_type)
    if not constructor:
        logger.warning(f"This block type constructor is not implemented: {block_type}")
        return None, None
    block = constructor(base_block)
    if getattr(block, "gerror", -1) != -1 or getattr(block, "gps_status", -1) == 0:
        logger.error("INFO", f"Block with error: {block_type}")
        return None, None  # spectral or gps blocks with error
    return block_type, block

A função a seguir recebe os bytes lidos do arquivo `.bin` e mapeia esses bytes em diferentes classes de acordo com o tipo de bloco

In [ ]:
#export
def parse_bin(bin_file: Union[str, Path], precision=np.float32) -> dict:
    """Receives a CRFS binfile and returns a dictionary with the file metadata, a GPS Class and a list with the different Spectrum Classes
    A block is a piece of the .bin file with a known start and end and that contains different types of information.
    It has several fields: file_type, header, data and footer.
    Each field has lengths and information defined in the documentation.
    Args:
        bin_file (Union[str, Path]): path to the bin file

    Returns:
        Dictionary with the file metadata, file_version, string info, gps and spectrum blocks.
    """
    bin_file = Path(bin_file)
    meta = {}
    fluxos = {}
    gps = CrfsGPS()
    with open(bin_file, mode="rb") as file:
        # The first block of the file is the header and is 36 bytes long.
        header = file.read(BYTES_HEADER)
        meta["filename"] = bin_file.name
        meta["file_version"] = bin2int(header[:4])
        meta["string"] = bin2str(header[4:])
        file_size = file.seek(0, 2)
        logger.info(f'Tamanho do arquivo: {file_size} bytes')
        file.seek(36, 0)
        while (next_block := file.tell()) < file_size:
            block_type, block = create_block(file, next_block)
            if (eof := file.read(4)) != b'UUUU':
                logger.error(f"EOF diferente de UUUU: {eof}, posicao: {file.tell()}")
                continue
            if block is None: 
                continue
            if block_type == 40:
                gps._data.append(block)
            elif block_type in VECTOR_BLOCKS:
                append_spec_data(block_type,fluxos, block, precision)
            else:
                meta.update(getattrs(block, KEY_ATTRS.get(block_type)))
    meta["gps"] = gps
    meta["spectrum"] = L(fluxos.values())
    return meta                 


In [ ]:
#exports
@dataclass
class CrfsGPS:
    """Class with the GPS Attributes from the CRFS Bin File"""

    _data: L = L()

    @cached
    def _gps_datetime(self):
        return self._data.attrgot("gps_datetime")

    @cached
    def _latitude(self):
        return self._data.attrgot("latitude")

    @cached
    def _longitude(self):
        return self._data.attrgot("longitude")

    @cached
    def _altitude(self):
        return self._data.attrgot("altitude")

    @cached
    def _num_satellites(self):
        return self._data.attrgot("num_satellites")

    @property
    def latitude(self) -> float:
        return np.median(self._latitude) if self._latitude else -1

    @property
    def longitude(self) -> float:
        return np.median(self._longitude) if self._longitude else -1

    @property
    def altitude(self) -> float:
        return np.median(self._altitude) if self._altitude else -1

    @property
    def num_satellites(self) -> float:
        return np.median(self._num_satellites) if self._num_satellites else 0

    def __repr__(self):
        return f"GPS Data - Median of Coordinates: {self.latitude:.5f}:{self.longitude:.5f} Altitude: {self.altitude:.2f} #Satellites: {self.num_satellites:.1f}"


class CrfsSpectrum(GetAttr):
    """Class with the metadata and levels of a spectrum block from a CRFS Bin File"""

    def __init__(self, metadata, precision=np.float32):
        self.default = metadata
        # self._timestamp: L = L()
        self._data: L = L()
        self.precision = precision

    def __len__(self):
        return len(self._data)

    def __repr__(self):
        return repr(self.default)

    def __str__(self):
        return f"""Blocks of Type: {self.type}, Thread_id: {self.thread_id}, Start: {self.start_mega} MHz, Stop: {self.stop_mega} MHz"""

    @cached
    def _timestamp(self):
        return self._data.attrgot('wallclock_datetime')

    @cached
    def start_dateidx(self):
        return getattr(self._data[0], 'wallclock_datetime').item()

    @cached
    def stop_dateidx(self):
        return getattr(self._data[-1], 'wallclock_datetime').item()

    @cached
    def levels(self):
        """Return the spectrum levels"""
        if self.type in UNCOMPRESSED:
            levels = np.empty((len(self._data), self.ndata), dtype=self.precision)
            for i, level in enumerate(self._data.attrgot('levels')):
                levels[i,:] = level
            # levels = np.concatenate(self._data.attrgot('levels')).reshape((-1, self.ndata))
        elif self.type in COMPRESSED:
            levels = cy_extract_compressed(
                list(self._data.attrgot('levels')),
                len(self._data),
                int(self.ndata),
                int(self.thresh),
                float(self.minimum),
            )
        else:
            raise ValueError(
                "The current block is not of type spectrum or it's not implemented yet"
            )
        if self.precision != np.float32:
            levels = levels.astype(self.precision)
        return levels

    @cached
    def frequencies(self) -> np.ndarray:
        return np.linspace(self.start_mega, self.stop_mega, num=self.ndata)

    def matrix(self):
        """Returns the matrix formed from the spectrum levels and timestamp"""
        index = self._timestamp if len(self._timestamp) == len(self) else None
        data = pd.DataFrame(self.levels, index=index, columns=self.frequencies)
        data.columns.name = "Frequencies"
        data.index.name = "Time"
        return data

In [ ]:
#export
def append_spec_data(block_type, fluxos, block, precision=np.float32) -> None:
    """Append the spectrum data to the fluxos dict"""
    keys, vals = getattrs(block, KEY_ATTRS.get(block_type), as_tuple=True)
    if vals not in fluxos:
        metadata = make_dataclass('SpecData', fields=[(k,type(k)) for k in keys])
        fluxos[vals] = CrfsSpectrum(metadata(*vals), precision)
    fluxos[vals]._data.append(block)

In [ ]:
file = get_files('binfiles/Erro')[0]
file

Path('binfiles/Erro/rfeye002213_CheckSum&EOF.bin')

In [ ]:
dados = parse_bin(file)
dados['spectrum']

(#22) [SpecData(type=67, thread_id=300, description='PMEC 2021 (Faixa 1 de 10).', start_mega=105, stop_mega=140, dtype='dBm', ndata=3584, bw=18457, processing='peak', antuid=0),SpecData(type=67, thread_id=310, description='PMEC 2021 (Faixa 2 de 10).', start_mega=155, stop_mega=165, dtype='dBm', ndata=1024, bw=18457, processing='peak', antuid=0),SpecData(type=67, thread_id=100, description='PRD 2021 (Faixa principal 1 de 4).', start_mega=50, stop_mega=90, dtype='dBμV/m', ndata=1024, bw=73828, processing='peak', antuid=0),SpecData(type=67, thread_id=110, description='PRD 2021 (Faixa principal 2 de 4).', start_mega=70, stop_mega=110, dtype='dBμV/m', ndata=1024, bw=73828, processing='peak', antuid=0),SpecData(type=67, thread_id=120, description='PRD 2021 (Faixa principal 3 de 4).', start_mega=170, stop_mega=220, dtype='dBμV/m', ndata=1280, bw=73828, processing='peak', antuid=0),SpecData(type=67, thread_id=130, description='PRD 2021 (Faixa principal 4 de 4).', start_mega=470, stop_mega=700,

In [ ]:
print(dados['spectrum'].attrgot('thread_id'))
print(dados['spectrum'].map(len))

[300, 310, 100, 110, 120, 130, 200, 210, 220, 230, 240, 250, 320, 330, 340, 350, 360, 370, 
380, 390, 310, 300]

[745, 744, 149, 149, 149, 149, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 1, 1]

In [ ]:
dados['spectrum'][0].matrix()

Frequencies,105.000000,105.009768,105.019537,105.029305,105.039073,105.048842,105.058610,105.068378,105.078147,105.087915,...,139.912085,139.921853,139.931622,139.941390,139.951158,139.960927,139.970695,139.980463,139.990232,140.000000
Time,,,,,,,,,,,,,,,,,,,,,
2021-08-21 12:03:01.699260,-88.5,-82.0,-73.5,-69.0,-69.5,-62.0,-50.5,-44.5,-42.0,-45.0,...,-100.5,-101.0,-106.0,-106.5,-108.0,-105.0,-102.0,-102.5,-105.0,-105.5
2021-08-21 12:04:01.979498,-74.0,-68.5,-68.0,-51.5,-50.0,-47.5,-45.0,-44.5,-46.0,-46.0,...,-99.5,-99.5,-99.0,-101.5,-106.0,-99.5,-101.0,-103.5,-100.5,-103.0
2021-08-21 12:05:01.919943,-79.0,-75.5,-70.5,-52.0,-46.5,-44.5,-45.5,-46.0,-41.5,-43.5,...,-95.5,-97.0,-92.0,-93.5,-98.0,-100.5,-105.5,-99.0,-101.0,-108.0
2021-08-21 12:06:01.349459,-93.0,-90.5,-81.0,-74.0,-67.5,-57.5,-48.0,-41.0,-39.5,-42.0,...,-94.0,-99.5,-101.5,-99.5,-104.0,-118.0,-107.0,-108.0,-105.0,-104.5
2021-08-21 12:07:01.279492,-85.0,-80.5,-74.0,-70.5,-63.0,-48.5,-41.5,-41.0,-42.5,-47.0,...,-96.5,-98.0,-101.0,-99.5,-101.0,-109.5,-101.5,-100.5,-101.0,-103.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-22 00:23:01.491219,-65.5,-59.5,-55.5,-54.5,-53.0,-47.5,-44.0,-48.0,-52.0,-52.5,...,-109.0,-105.0,-103.0,-99.5,-101.0,-104.0,-104.5,-105.5,-106.5,-98.0
2021-08-22 00:24:01.481216,-64.5,-63.0,-59.0,-50.5,-45.0,-43.5,-47.0,-53.5,-54.0,-62.5,...,-105.5,-104.0,-104.0,-99.5,-97.5,-97.5,-105.5,-105.5,-107.0,-104.5
2021-08-22 00:25:01.651117,-103.5,-99.5,-92.5,-88.0,-81.5,-75.5,-70.5,-61.5,-58.5,-56.5,...,-104.0,-112.0,-118.0,-109.0,-108.5,-107.5,-105.0,-102.0,-104.0,-100.0


In [ ]:
%%cython --annotate
cimport cython

ctypedef object CrfsGPS

@cython.boundscheck(False)
@cython.wraparound(False)



Error compiling Cython file:
------------------------------------------------------------
...

ctypedef object CrfsGPS

@cython.boundscheck(False)
@cython.wraparound(False)
^
------------------------------------------------------------

C:\Users\rsilva\.ipython\cython\_cython_magic_98c655412a46cff311afb5f5b8b31186.pyx:7:0: Decorators can only be followed by functions or classes
